In [ ]:
import numpy as np
from pprint import pprint
np.set_printoptions(threshold=np.inf)

In [ ]:
sudoku = np.zeros((4,4))
HansonMatrix = np.zeros((729,324))
sudoku = np.array([[1,2,3,4],[3,4,1,2],[4,1,2,3],[2,3,4,1]])
# grid = np.array([[4,2,9,0,0,3,0,0,0],
#                 [3,7,0,1,0,0,9,0,0],
#                 [0,0,0,2,9,0,3,0,0],
#                 [8,3,0,0,1,0,4,0,6],
#                 [0,0,0,8,0,0,7,5,0],
#                 [7,4,5,9,3,0,0,1,0],
#                 [9,0,0,7,0,0,0,3,1],
#                 [2,5,7,0,0,0,6,0,0],
#                 [6,0,3,0,0,9,8,0,7]])
#pprint(HansonMatrix)

Rows - Every Consecutive 4 rows represent a cell. R1C1 -> R1C2 - > R1C3 -> ...... ; 16 cells in total. 64 rows.

Columns - 

First 16 columns - Row Column Constraints. R1C1 -> R1C2 -> R1C3 -> R1C4 ...... ; Means only one number per cell.
Second 16 columns - Row & Number Constaints. R1#1 -> R1#2 -> R1#3 ....... ; Means a number only once in a row.
Third 16 columns - Column & Number Constraints. C1#1 -> C1#2 -> C1#3 ....... ; Means a number only once in a column.
Fourth 16 columns - Box and Number Constraints. B1#1 -> B1#2 ......... ; Means a number only once in a box.

In [ ]:

class Node:
    def __init__(self,row = None, column = None):
        self.left = self
        self.right = self
        self.up = self
        self.down = self
        self.header = self
        self.row = row
        self.column = column
        self.nodeCount = 0

In [ ]:
def rowNodes(node):
    list = []
    list.append(node)
    a = node.column
    current = node.right
    while (current.column != a):
        list.append(current)
        current = current.right
    return list

def columnNodes(node):
    list = []
    a = node.row
    #print(a)
    current = node.up
    #print(current.row)
    while (current.row != a):
        if (current.row == -1):
            current = current.up
            continue
        list.append(current.row)
        current = current.up
    
    return list

def initialize(HansonMatrix,numbers):

    for column in range(numbers*numbers):
        for row in range(column*numbers,column*numbers + numbers):
            HansonMatrix[row][column] = 1
    
    for i in range(numbers):
        for chunk in range(i*numbers, (i+1)*numbers):
            for j in range(numbers):
                row = chunk*numbers + j
                column = i*numbers + j + numbers*numbers
                HansonMatrix[row][column] = 1

    for i in range(numbers):
        for j in range(numbers*numbers):
            row = i*numbers*numbers + j
            column = j + 2*numbers*numbers
            HansonMatrix[row][column] = 1

    for i in range(np.sqrt(numbers).astype(int)):
        for j in range(np.sqrt(numbers).astype(int)):
            for k in range(np.sqrt(numbers).astype(int)):
                for w in range(np.sqrt(numbers).astype(int)):
                    for r in range(numbers):
                        column = 3*numbers*numbers + i*numbers*np.sqrt(numbers).astype(int) + k*numbers + r
                        row = i*numbers*numbers*np.sqrt(numbers) + j*numbers*numbers + k*numbers*np.sqrt(numbers).astype(int) + w*numbers + r
                        #print(f"{row}" + " " +  f"{column}")
                        row = row.astype(int)
                        column = column.astype(int)
                        #print(row)
                        HansonMatrix[row][column] = 1

def initial_links(HansonMatrix,HeaderList,RowList):

    rows = len(HansonMatrix)
    columns = len(HansonMatrix[0])
    for i in range(rows):
        prevNode = None
        firstNode = None
        
        for j in range(columns):
            if HansonMatrix[i][j]:
                HeaderList[j].nodeCount += 1
                newNode = Node(row=i,column=j)

                # print(newNode.row)
                # print(newNode.column)

                if not firstNode:
                    firstNode = newNode

                newNode.header = HeaderList[j]
                
                if prevNode:
                    newNode.left = prevNode
                    prevNode.right = newNode

                newNode.up = HeaderList[j].up
                HeaderList[j].up.down = newNode
                HeaderList[j].up = newNode
                newNode.down = HeaderList[j]
                prevNode = newNode

                if HeaderList[j].down == HeaderList[j]:
                    HeaderList[j].down = newNode
                
                # column_header = HeaderList[j]
                # print(f"Column {j} nodes:")
                # current_node = column_header.down
                # while current_node != column_header:
                #     print(f"  Node at row {current_node.row}: up -> {current_node.up.row}, down -> {current_node.down.row}")
                #     current_node = current_node.down

        if firstNode and prevNode:
            prevNode.right = firstNode
            firstNode.left = prevNode
        if firstNode:
            RowList.append(firstNode)
    

    # for j in range(columns):
    #     print(f"Header {j}: left -> {HeaderList[j].left}, right -> {HeaderList[j].right}")
    
    # # Check nodes in each column
    # for j in range(columns):
    #     column_header = HeaderList[j]
    #     print(f"Column {j} nodes:")
    #     current_node = column_header.down
    #     while current_node != column_header:
    #         print(f"  Node at row {current_node.row}: up -> {current_node.up.row}, down -> {current_node.down.row}")
    #         current_node = current_node.down
            
def fitting_sudoku(HansonMatrix,grid,RowList,numbers):
    cleanup = []
    rows = len(grid)
    columns = len(grid[0])
    for i in range(rows):
        for j in range(columns):
            value = grid[i][j]
            if value == 0:
                continue
            row_indice = numbers*numbers*i + numbers*j + value - 1
            current = RowList[row_indice]
            nodes = rowNodes(current)
            for node in nodes:
                #print(node.row)
                cleanup += columnNodes(node)
                #print(columnNodes(node))
    
    clear = list(set(cleanup))
    #print(clear)
    for i in clear:
        HansonMatrix[i,:] = 0

def sudoku_links(HansonMatrix,grid,RowList,HeaderList):
    initial_links(HansonMatrix,HeaderList,RowList)
    fitting_sudoku(HansonMatrix,grid,RowList)

def printing(HeaderList):
    for node in HeaderList:
        current = node.down
        while (current != node):
            print(current.row)
            print(current.column)

def cover(Node):
    header = Node.header
    header.left.right = header.right
    header.right.left = header.left

    current = Node.down

    while current != header:
        right = current.right
        while right != current:
            right.up.down = right.down
            right.down.up = right.up
            right.header.nodeCount -= 1
            right = right.right
        current = current.down

def uncover(Node):
    header = Node.header

    up = header.up
    while up != header:
        left = up.left
        while left != up:
            left.up.down = left
            left.down.up = left
            left.header.nodeCount += 1
            left = left.left
        up = up.up

    header.left.right = header
    header.right.left = header

def minColumn(HeaderList):
    current = HeaderList[0]
    a = current.nodeCount
    for node in HeaderList:
        if node.nodeCount < a:
            a = node.nodeCount
            current = node
    return current

def print_sol(solutions):
    for node in solutions:
        print(f"{node.row} +  {node.column}")
        #print(node.column)

def solve(k,HeaderList,solutions,centralNode,RowList):
    
    if centralNode.right == centralNode:
        print_sol(solutions)
        return
    
    column = minColumn(HeaderList)
    cover(column)

    # print("Checking first cover")
    # print_all_nodes(centralNode)
    # print_rows(RowList)

    row = column.down

    while row != column:

        solutions.append(row)

        right = row.right
        while right != row:
            print(f"{right.row} + {right.column}")
            print("HEREEEE")
            cover(right)
            right = right.right

        solve(k+1,HeaderList,solutions,centralNode,RowList)
        solutions.pop()
        column = row.column

        left = row.left
        while left != row:
            ("HERASDFAD")
            uncover(left)
            left = left.left

        row = row.down
    
    uncover(column)

def print_all_nodes(node):
    if not node:
        return
    
    print("Printing all nodes in the structure column wise:")
    start_node = node
    
    # Start by traversing the columns
    col_node = start_node
    while True:
        print(f" Node of Interest -    Row: {col_node.row}, Column: {col_node.column}")
        print ("now going downwards and checking this column")
        
        # Now traverse the rows for this column
        row_node = col_node.down
        while row_node != col_node:
            print(f"    Row: {row_node.row}, Column: {row_node.column}")
            row_node = row_node.down
        
        col_node = col_node.right
        if col_node == start_node:
            break

def print_rows(RowList):
    for node in RowList:
        print("Printing all nodes in the structure row wise:")
        start_node = node
    
        # Start by traversing the rows
        row_node = start_node
        while True:
            print(f" Node of Interest -    Row: {row_node.row}, Column: {row_node.column}")
            print ("now going sideways and checking this row")
            
            # Now traverse the rows for this column
            col_node = row_node.right
            while col_node != row_node:
                print(f"    Row: {col_node.row}, Column: {col_node.column}")
                col_node = col_node.right
            
            if row_node == col_node:
                break

In [ ]:
HansonMatrix = np.zeros((64,64))
numbers = 4
grid = np.array([[1,2,0,4],[3,0,1,2],[4,0,2,0],[0,3,0,1]])
RowList = []

columns = 4*numbers*numbers
rows = numbers*numbers*numbers

HeaderList = []
HeaderList = [Node(row = -1, column= _) for _ in range(columns)]
centralNode = Node(row=-1, column=-1)

RowList = []

HeaderList[0].left = centralNode
centralNode.right = HeaderList[0]
HeaderList[0].right = HeaderList[1]
HeaderList[columns-1].right = centralNode
centralNode.left = HeaderList[columns-1]
HeaderList[columns-1].left = HeaderList[columns - 2]

for j in range(1,columns-1):
    HeaderList[j].left = HeaderList[j-1]
    HeaderList[j].right = HeaderList[j+1]
    
print_all_nodes(centralNode)


In [ ]:
initialize(HansonMatrix,numbers)
initial_links(HansonMatrix,HeaderList, RowList)

print_all_nodes(centralNode)
print_rows(RowList)

In [ ]:

fitting_sudoku(HansonMatrix,grid,RowList,numbers)

print_all_nodes(centralNode)
print_rows(RowList)

In [ ]:

HeaderList = []
HeaderList = [Node(row = -1, column= _) for _ in range(columns)]

RowList = []
centralNode = Node(row=-1, column=-1)

HeaderList[0].left = centralNode
centralNode.right = HeaderList[0]
HeaderList[0].right = HeaderList[1]
HeaderList[columns-1].right = centralNode
centralNode.left = HeaderList[columns -1]
HeaderList[columns-1].left = HeaderList[columns - 2]

for j in range(1,columns-1):
    HeaderList[j].left = HeaderList[j-1]
    HeaderList[j].right = HeaderList[j+1]

initial_links(HansonMatrix,HeaderList,RowList)

print_all_nodes(centralNode)
print_rows(RowList)

solutions = []


#sudoku_links(HansonMatrix,grid,RowList,HeaderList)

In [ ]:
solve(0,HeaderList,solutions,centralNode,RowList)